In [1]:
import os

# 获取当前工作目录
current_dir = os.getcwd()
print("当前工作目录：", current_dir)

当前工作目录： /mnt/e/0_Projects/基于集成学习的恶意PE软件特征检测与识别/mal_ana/machine_learning/ensemble_learning/experiment3


In [2]:
# 设置新的工作目录
os.chdir('/mnt/e/0_Projects/基于集成学习的恶意PE软件特征检测与识别/mal_ana/machine_learning/ensemble_learning')

# 获取更新后的工作目录
updated_dir = os.getcwd()
print("更新后的工作目录：", updated_dir)

更新后的工作目录： /mnt/e/0_Projects/基于集成学习的恶意PE软件特征检测与识别/mal_ana/machine_learning/ensemble_learning


In [3]:
%reload_ext autoreload
%autoreload 2
import os
import sys
import time
import argparse
import numpy as np
import pandas as pd
import pickle

from lgbm_model import load_data, train_lightgbm_model, train_pca_lightgbm_model
from feature_analysis import (plot_feature_importance, plot_roc_and_pr_curves, 
                              plot_model_comparison, plot_pca_analysis, 
                              plot_data_distribution)
from model_evaluation import (compare_models, analyze_feature_robustness, 
                             measure_inference_time)
from constants import create_directories

In [4]:
import imp
import lgbm_model
import feature_analysis
import model_evaluation
import constants
imp.reload(lgbm_model)
imp.reload(feature_analysis)
imp.reload(model_evaluation)
imp.reload(constants)
from lgbm_model import load_data, train_lightgbm_model, train_pca_lightgbm_model
from feature_analysis import (plot_feature_importance, plot_roc_and_pr_curves, 
                              plot_model_comparison, plot_pca_analysis, 
                              plot_data_distribution)
from model_evaluation import (compare_models, analyze_feature_robustness, 
                             measure_inference_time)
from constants import create_directories

In [5]:
np.random.seed(42)
result_dir = './experiment3/results'
figure_dir = './experiment3/figures'

In [6]:
# 1. 加载数据
print("=" * 80)
print("1. 加载数据")
print("=" * 80)
features, labels, feature_names = load_data()

1. 加载数据
加载数据集...
数据集加载完成，特征维度：(17657, 56)，标签数量：17657
标签分布：良性样本 = 11759，恶意样本 = 5898


In [7]:
# 2. 训练基础LightGBM模型
print("\n" + "=" * 80)
print("2. 训练基础LightGBM模型")
print("=" * 80)
models, feature_importance_df, metrics_results, test_data = train_lightgbm_model(
    features, labels, feature_names, 
    save_dir=os.path.join(result_dir, 'lightgbm')
)
X_test, y_test, y_pred = test_data


2. 训练基础LightGBM模型
训练集特征形状: (14125, 56), 标签形状: (14125,)
测试集特征形状: (3532, 56), 标签形状: (3532,)

开始5折交叉验证训练...

训练折 1/5...
训练LightGBM模型...设置early_stopping_rounds=50
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[50]	train's auc: 0.999704	valid's auc: 0.995275

训练折 2/5...
训练LightGBM模型...设置early_stopping_rounds=50
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[49]	train's auc: 0.999661	valid's auc: 0.996804

训练折 3/5...
训练LightGBM模型...设置early_stopping_rounds=50
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[49]	train's auc: 0.999716	valid's auc: 0.996733

训练折 4/5...
训练LightGBM模型...设置early_stopping_rounds=50
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[50]	train's auc: 0.999707	valid's auc: 0.994863

训练折 5/5...
训练LightGBM模型...设置early_stopping_ro

In [8]:
# 3. 绘制基础模型的特征重要性和性能图表
print("\n" + "=" * 80)
print("3. 绘制基础模型的特征重要性和性能图表")
print("=" * 80)
# 绘制特征重要性
plot_feature_importance(
    feature_importance_df, 
    save_dir=figure_dir, 
    top_n=20
)
# 绘制ROC和PR曲线
plot_roc_and_pr_curves(
    y_test, y_pred, 
    save_dir=figure_dir
)

# 绘制数据分布
plot_data_distribution(
    features, labels, feature_names, 
    save_dir=figure_dir, 
    top_n=5
)


3. 绘制基础模型的特征重要性和性能图表
Top 20 特征重要性图表已保存
ROC和PR曲线图表已保存
执行PCA降维...
数据分布图表已保存


In [9]:
# 4. 训练PCA降维的LightGBM模型并比较
print("\n" + "=" * 80)
print("4. 训练PCA降维的LightGBM模型并比较")
print("=" * 80)
pca_models, pca_metrics, pca_test_data, pca = train_pca_lightgbm_model(
    features, labels, feature_names, 
    n_components=10, 
    save_dir=os.path.join(result_dir, 'lightgbm_pca')
)

# 绘制PCA分析图
plot_pca_analysis(
    pca, feature_names, 
    save_dir=figure_dir
)


4. 训练PCA降维的LightGBM模型并比较

开始训练基于PCA降维(保留10个主成分)的LightGBM模型...
数据中包含NaN: True
数据中包含无穷值: False
正在处理异常值...
替换 5419 个NaN值为0
清洗后数据范围: [0.0, 1275061248.0]
PCA降维后特征维度: (17657, 10)
累计解释方差比: 0.5848
训练集特征形状: (14125, 10), 标签形状: (14125,)
测试集特征形状: (3532, 10), 标签形状: (3532,)

开始3折交叉验证训练...

训练折 1/3...
训练LightGBM模型...设置early_stopping_rounds=50
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[50]	train's auc: 0.998459	valid's auc: 0.989547

训练折 2/3...
训练LightGBM模型...设置early_stopping_rounds=50
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[50]	train's auc: 0.998157	valid's auc: 0.991481

训练折 3/3...
训练LightGBM模型...设置early_stopping_rounds=50
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[50]	train's auc: 0.998631	valid's auc: 0.988825

模型训练完成！总训练时间: 7.93秒 (0.13分钟)

评估模型性能...

验证集性能指标:
准确率 (Accuracy): 0.9623
精确率 (Precisio

In [10]:
# 5. 比较不同模型性能
print("\n" + "=" * 80)
print("5. 比较不同模型性能")
print("=" * 80)
comparison_df, test_data_dict = compare_models(
    features, labels, feature_names, 
    save_dir=result_dir
)

# 绘制模型比较图
plot_model_comparison(
    comparison_df, 
    save_dir=figure_dir
)


5. 比较不同模型性能

开始比较不同模型性能...

清洗数据中的异常值...
数据中包含NaN: True
数据中包含无穷值: False
正在清洗数据中的异常值...
替换 5419 个NaN值为0
清洗后数据范围: [0.0, 1275061248.0]

1. 训练LightGBM模型（原始特征）
训练集特征形状: (14125, 56), 标签形状: (14125,)
测试集特征形状: (3532, 56), 标签形状: (3532,)

开始5折交叉验证训练...

训练折 1/5...
训练LightGBM模型...设置early_stopping_rounds=50
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[50]	train's auc: 0.999677	valid's auc: 0.994751

训练折 2/5...
训练LightGBM模型...设置early_stopping_rounds=50
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[49]	train's auc: 0.999655	valid's auc: 0.996733

训练折 3/5...
训练LightGBM模型...设置early_stopping_rounds=50
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[41]	train's auc: 0.999549	valid's auc: 0.996472

训练折 4/5...
训练LightGBM模型...设置early_stopping_rounds=50
Training until validation scores don't improve for 50 rounds
Did no

In [11]:
# 6. 分析模型鲁棒性和推理时间
print("\n" + "=" * 80)
print("6. 分析模型鲁棒性和推理时间")
print("=" * 80)

# 获取第一个LightGBM模型进行鲁棒性分析
lgbm_model = models[0]

# 分析特征鲁棒性
robustness_df = analyze_feature_robustness(
    lgbm_model, X_test, y_test, feature_names, 
    perturbation_ratio=0.1, 
    n_runs=5, 
    save_dir=os.path.join(result_dir, 'robustness')
)

# 构建模型字典用于推理时间测量
models_dict = {
    'LightGBM': lgbm_model
}


6. 分析模型鲁棒性和推理时间

开始特征鲁棒性分析...
使用predict方法预测...
基准性能: AUC = 0.9966, Accuracy = 0.9773
运行 1/5...
运行 2/5...
运行 3/5...
运行 4/5...
运行 5/5...

特征鲁棒性分析结果:
       run       auc  accuracy  auc_drop  accuracy_drop
0        1  0.996357  0.976784  0.000248       0.000566
1        2  0.935819  0.732729  0.060785       0.244621
2        3  0.996048  0.971687  0.000556       0.005663
3        4  0.996206  0.974802  0.000399       0.002548
4        5  0.992501  0.969706  0.004104       0.007644
5  Average  0.983386  0.925142  0.013218       0.052208
平均AUC下降: 0.0132
平均Accuracy下降: 0.0522


In [12]:
# 读取其他模型（如果存在）
xgb_path = os.path.join(result_dir, 'xgboost', 'xgb_model.pkl')
rf_path = os.path.join(result_dir, 'random_forest', 'rf_model.pkl')

if os.path.exists(xgb_path):
    with open(xgb_path, 'rb') as f:
        models_dict['XGBoost'] = pickle.load(f)

if os.path.exists(rf_path):
    with open(rf_path, 'rb') as f:
        models_dict['RandomForest'] = pickle.load(f)

# 测量推理时间
inference_time_df = measure_inference_time(
    models_dict, X_test, 
    n_repeats=50, 
    save_dir=os.path.join(result_dir, 'inference_time')
)

# 返回结果字典
# return {
#     'feature_importance': feature_importance_df,
#     'metrics': metrics_results,
#     'comparison': comparison_df,
#     'robustness': robustness_df,
#     'inference_time': inference_time_df
# }



开始测量模型推理时间...
清洗输入数据...

测量 LightGBM 模型的推理时间...

测量 XGBoost 模型的推理时间...

测量 RandomForest 模型的推理时间...

模型推理时间测量完成，结果已保存
